In [3]:
from tqdm import tqdm
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import pandas as pd

In [4]:
# Create a new Chrome session
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode

driver = webdriver.Chrome()
driver.get('https://www.amazon.in')

In [5]:
# Manually input product URLs
product_urls = [
    'https://www.amazon.in/Kurl-Mattress-Orthopedic-72x48x5-Ortho/dp/B0BNJ42DF9/ref=sr_1_1_sspa?crid=1YAZ4LZJ2G62X&dib=eyJ2IjoiMSJ9.xbwEKkpazp59E8btcje8ESXiNckXfcVPiWE9BrxdGXLIn9NkNgds-NNQkZ9qZtiD-Bzn5EaDvHM3HhmhGL63lUHEi93ikLqBPFKnjx3NesK1FszbsRrWm6tG7CH8E-c-U6yphOrMvFb-uAXFrozuY3CutEmw7E1gybjdRQhFeuoZUsdRoP0M6s6Hc2qQgNQnYp_UD2B9_8SbrcIhxeS7TBnlce3nUP61zG3pYQhV5zo77Bg5icRdUZ_1UqTpXncT1ag-VzPo0oM_iDsfDY2QQu6N2y9wHyx4BbbtqTMmLaw.mrUlrFLC6Rg07WgM-Nb0RdsuFOjUQzYFRM8d-v2TY3Y&dib_tag=se&keywords=sleepyhug+ortho+spinex+mattress&qid=1710754378&sprefix=sleephug+ortho+spinex+mattress%2Caps%2C204&sr=8-1-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&psc=1',
    'https://www.amazon.in/Sleep-Company-SmartGRID-Orthopedic-Pro/dp/B0BPMJGQXV/ref=sr_1_4_sspa?crid=1YAZ4LZJ2G62X&dib=eyJ2IjoiMSJ9.xbwEKkpazp59E8btcje8ESXiNckXfcVPiWE9BrxdGXLIn9NkNgds-NNQkZ9qZtiD-Bzn5EaDvHM3HhmhGL63lUHEi93ikLqBPFKnjx3NesK1FszbsRrWm6tG7CH8E-c-U6yphOrMvFb-uAXFrozuY3CutEmw7E1gybjdRQhFeuoZUsdRoP0M6s6Hc2qQgNQnYp_UD2B9_8SbrcIhxeS7TBnlce3nUP61zG3pYQhV5zo77Bg5icRdUZ_1UqTpXncT1ag-VzPo0oM_iDsfDY2QQu6N2y9wHyx4BbbtqTMmLaw.mrUlrFLC6Rg07WgM-Nb0RdsuFOjUQzYFRM8d-v2TY3Y&dib_tag=se&keywords=sleepyhug+ortho+spinex+mattress&qid=1710754378&sprefix=sleephug+ortho+spinex+mattress%2Caps%2C204&sr=8-4-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&psc=1',
    'https://www.amazon.in/SleepyHug-Orthopedic-Mattress-Reversible-Tape-Edge/dp/B0CJDXTC7T/ref=sr_1_9?sr=8-9',
    # Add more product URLs as needed
]

In [6]:
titles = []
ratings = []
prices = []
mrps = []
discounts = []

for url in tqdm(product_urls):
    driver.get(url)
    time.sleep(3)
    html_data = BeautifulSoup(driver.page_source, 'html.parser')
    product = html_data.find('div', {'id': 'ppd'})
    if product:
        # Extract title
        title = product.find('span',{'class':'a-size-large product-title-word-break'}).text
        titles.append(title)
        
        #Extract ratings
        rating = product.find('span',{'class':'a-icon-alt'})
        if rating  == None:
            rating = 'No Rating'
        else:
            rating = product.find('span',{'class':'a-icon-alt'}).text
        ratings.append(rating)
        
        #extract price
        price_element = product.find('span', {'class': 'a-price-whole'})
        if price_element is not None:
            price = price_element.text
            prices.append(price)
        else:
            prices.append('N/A')  # or any other value you want to assign when price is not found
        
      # Extract mrp
        mrp_element = product.find('span', {'class': 'a-price a-text-price'})
        if mrp_element is not None:
            # Extracting the price from the first span element within the parent span
            price_span = mrp_element.find('span', {'aria-hidden': 'true'})
            if price_span is not None:
                mrp = price_span.text.strip()
                # Remove the '₹' symbol from the MRP text
                mrp = mrp.replace('₹', '').strip()
                mrps.append(mrp)
            else:
                # If price is not found within the nested span, extract from the offscreen span
                offscreen_span = mrp_element.find('span', {'class': 'a-offscreen'})
                if offscreen_span is not None:
                    mrp = offscreen_span.text.strip()
                    # Remove the '₹' symbol from the MRP text
                    mrp = mrp.replace('₹', '').strip()
                    mrps.append(mrp)
                else:
                    mrps.append('N/A')  # Append 'N/A' if price is not found
        else:
            mrps.append('N/A')  # Append 'N/A' if no price element is found


        #extract discount
        discount_element = product.find('span', {'class': 'a-size-medium a-color-price savingPriceOverride aok-align-center reinventPriceSavingsPercentageMargin savingsPercentage'})
        if discount_element is not None:
            discount_percentage = discount_element.text.strip()
        else:
            discount_percentage = 'N/A'
        discounts.append(discount_percentage)

# Print or process the scraped data
print("Titles:", titles)
print("Ratings:", ratings)
print("Prices:", prices)
print("MRPs:", mrps)
print("Discounts:", discounts)

100%|██████████| 3/3 [00:22<00:00,  7.50s/it]

Titles: ['        Kurl-On Mattress |Spinel Ortho|Orthopedic Mattress|5-Inch Double Size Mattress |72x48x5 Inches Firm Support | Bonded Foam Mattres.       ', '        The Sleep Company SmartGRID Orthopedic Pro - Doctor Recommended | Pressure Relieving | Scientifically Proven 5 Zone Support | 6 Inch Single Size AIHA Certified Firm Mattress for Back Pain | 75x36       ', '        SleepyHug AirCell Ortho Dual Comfort Plus Orthopedic Mattress with Reversible Hard & Soft Foam 6 inch Mattress AirCell Tech, Ideal for Single Bed, Back Pain Relief, Tape-Edge Cover, (78x36x6)       ']
Ratings: ['4.0 out of 5 stars', '4.6 out of 5 stars', '4.6 out of 5 stars']
Prices: ['9,547.', '15,500.', '7,340.']
MRPs: ['12,399', '25,833', '8,882']
Discounts: ['N/A', 'N/A', 'N/A']


In [7]:
data = pd.DataFrame({
    'titles': titles,
    'ratings': ratings,
    'prices': prices,
    'mrps': mrps,
    'discounts': discounts
})
data

,titles,ratings,prices,mrps,discounts
0,Kurl-On Mattress |Spinel Ortho|Orthope...,4.0 out of 5 stars,"9,547.","12,399",N/A
1,The Sleep Company SmartGRID Orthopedic...,4.6 out of 5 stars,"15,500.","25,833",N/A
2,SleepyHug AirCell Ortho Dual Comfort P...,4.6 out of 5 stars,"7,340.","8,882",N/A


In [8]:
csv_file_path = '/home/user/Data/scrapping/items.csv'
data.to_csv(csv_file_path, index=True)
print("CSV file saved successfully.")

CSV file saved successfully.
